# Spatial Optimization

In [10]:
import folium
import pandas as pd
import numpy as np
import geopandas as gpd
from pyproj import CRS
from shapely.geometry import Point
from sklearn.neighbors import KernelDensity
from statsmodels.nonparametric.bandwidths import bw_silverman

## Configure Parameters

In [11]:
bus_stops_weight = 0.5
hospitals_weight = 1.0
restaurants_weight = 0.25
num_bpoints = 5  # number of best points required

## Load Data

In [12]:
bus_stops_file = './bus_stops.csv'
hospitals_file = './hospitals.csv'
restaurants_file = './restaurants.csv'
bus_stops_data = pd.read_csv(bus_stops_file)
hospitals_data = pd.read_csv(hospitals_file)
restaurants_data = pd.read_csv(restaurants_file)

## Prepare Data

In [13]:
bus_stops_data['weights'] = bus_stops_weight
bus_stops_data['location_type'] = 'bus_stop'
hospitals_data['weights'] = hospitals_weight
hospitals_data['location_type'] = 'hospital'
restaurants_data['weights'] = restaurants_weight
restaurants_data['location_type'] = 'restaurant'
datasets = [bus_stops_data, hospitals_data, restaurants_data]
merged_data = pd.concat(datasets)
weights = merged_data['weights']
display(merged_data.head())

,id,latitude,longitude,weights,location_type
0,d461dd24beb4b0ef4eb8f5251f14a133,12.955712,77.720346,0.5,bus_stop
1,567e9aa5e9849f42f6161bf6b73bd47c,12.846598,77.671565,0.5,bus_stop
2,cf2fd7e978ea8f70b790c931cb6c8ccf,12.939330,77.695316,0.5,bus_stop
3,ca41840db8acbbc8a10e6e4c69d91c36,12.951165,77.491945,0.5,bus_stop
4,219773e23e2d7659a5935bb63e75347f,13.197742,77.777031,0.5,bus_stop


## Kernel Density Estimation

In [14]:
silverman_bandwidth = max(bw_silverman(
    merged_data[['latitude', 'longitude']].to_numpy()))
kde = KernelDensity(bandwidth=silverman_bandwidth,
                    algorithm='ball_tree', metric='haversine')
trained_estimator = kde.fit(
    merged_data[['latitude', 'longitude']].to_numpy(), weights.to_numpy())

## Finding Best Coordinates

In [15]:
bpoint_indices = np.argsort(trained_estimator.score_samples(
    merged_data[['latitude', 'longitude']].to_numpy()))[-num_bpoints:]
bpoint_indices = np.flip(bpoint_indices)  # order descendingly
best_locations = merged_data.iloc[bpoint_indices]
best_locations.index = range(1, 6)  # show the ranks of the locations
display(best_locations)

,id,latitude,longitude,weights,location_type
1,b135301eceaf851c9c7c3efbd4de5ec0,12.957070,77.567107,0.50,bus_stop
2,71f1c68856adbc9e1b25010a0d1a5415,12.957209,77.568077,0.50,bus_stop
3,bba72a25200bebff3f1543d3ca700a16f542c3f6,12.953921,77.568874,0.25,restaurant
4,99d2b60af8c8e1340de223d22b7d0f56,12.957195,77.568295,0.50,bus_stop
5,76ab28bae03e2d48037ae3cefd48565e,12.960405,77.567908,0.50,bus_stop


## Visualization

In [16]:
# convert locations to points ('x' is longitude and 'y' is latitude)
best_points = [Point(location) for location in zip(
    best_locations['longitude'], best_locations['latitude'])]
gdf = gpd.GeoDataFrame({
    'id': best_locations['id'].to_numpy(),
    'location_type': best_locations['location_type'].to_numpy(),
    'geometry': best_points
})  # geo dataframe

# set crs - coordinate reference system
gdf.crs = CRS.from_epsg(4326)  # latitude longitude system
# convert to mercator system because our map is a mercator map
gdf.to_crs(CRS.from_epsg(3395), inplace=True)

# map it
map_plot = folium.Map(location=[
    np.mean(best_locations['latitude'].to_numpy()),
    np.mean(best_locations['longitude'].to_numpy())], zoom_start=14)
points_gjson = folium.features.GeoJson(gdf['geometry'], name='best_locations')
points_gjson.add_to(map_plot)
map_plot